# Useful stuff: CV

## Загрузка данных
#### Данные беру из Churn_modeling.zip

# Colab

https://drive.google.com/file/d/1ilb0iIuToOiuFSQpfWQvTCG4jl8I6hOx/view?usp=sharing

In [1]:
! gdown --id 1ilb0iIuToOiuFSQpfWQvTCG4jl8I6hOx

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


! unzip /content/churn_modeling.zip -d /content/churn_modeling/

In [9]:
! unzip / content / churn_modeling.zip -d / content / churn_modeling /

"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/churn_modeling/Churn_Modelling.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/churn_modeling/Churn_Modelling.csv'

In [ ]:
df.head()

# Pycharm

In [1]:
from zipfile import ZipFile

with ZipFile(r'C:\PyCharm\Python\Data_Science_GM\7.Classification\churn_modeling.zip', 'r') as zf:
    zf.extractall()

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Обучающая, валидационная и тестовая выборки

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(df, random_state=42, train_size=0.6, stratify=df.Exited)

In [6]:
val, test = train_test_split(test, random_state=42, train_size=0.5, stratify=test.Exited)

In [9]:
test.Exited.head()

9167    0
9387    0
6859    0
3500    0
6233    0
Name: Exited, dtype: int64

## Machine learning

In [7]:
# Для Colab
# ! pip install catboost

In [8]:
from catboost import CatBoostClassifier

In [9]:
X = ['CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
     'IsActiveMember', 'EstimatedSalary', 'Geography', 'Gender']
cat_features = ['Geography', 'Gender']

y = ['Exited']

In [10]:
from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                  )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                  )
test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                  )

In [18]:
params = {'verbose': 100,
          'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          'random_seed': 42,
          'learning_rate': 0.01}

In [19]:
model = CatBoostClassifier(**params)

In [20]:
%%time
model.fit(train_data, eval_set=valid_data)

0:	test: 0.8338545	best: 0.8338545 (0)	total: 20.4ms	remaining: 20.3s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 2.71s	remaining: 24.1s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 5.25s	remaining: 20.9s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 7.77s	remaining: 18s
400:	test: 0.8823868	best: 0.8824376 (390)	total: 10.3s	remaining: 15.4s
500:	test: 0.8834491	best: 0.8834522 (498)	total: 12.9s	remaining: 12.9s
600:	test: 0.8834737	best: 0.8836554 (564)	total: 15.4s	remaining: 10.2s
700:	test: 0.8837847	best: 0.8838694 (685)	total: 17.8s	remaining: 7.58s
800:	test: 0.8834383	best: 0.8838694 (685)	total: 20s	remaining: 4.97s
900:	test: 0.8829318	best: 0.8838694 (685)	total: 22.4s	remaining: 2.46s
999:	test: 0.8822744	best: 0.8838694 (685)	total: 24.8s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685

Shrink model to first 686 iterations.
CPU times: total: 54.1 s
Wall time: 25 s


------------------------------------------------

Обучение на всех данных

In [15]:
n_iters = model.best_iteration_ + 1

In [16]:
params = {'verbose': n_iters,
          'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          'random_seed': 42,
          'learning_rate': 0.01}

In [17]:
model = CatBoostClassifier(**params)

Объединение трейн и вал

In [18]:
train_full = pd.concat([train, val])

In [19]:
train_full_data = Pool(train_full[X],
                       label=train_full[y],
                       cat_features=cat_features)

In [20]:
model.fit(train_full_data)

0:	total: 23.5ms	remaining: 23.4s
686:	total: 16.2s	remaining: 7.39s
999:	total: 22.8s	remaining: 0us


In [21]:
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1]

In [22]:
from sklearn.metrics import roc_auc_score

In [23]:
roc_auc_score(test.Exited, test['y_score_no_cross_val'])

0.8719335668488212

## Кросс-валидация

https://github.com/catboost/tutorials/blob/master/classification/classification_tutorial.ipynb

In [24]:
from catboost import cv

In [26]:
params = {'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          'random_seed': 42,
          'learning_rate': 0.01}

In [27]:
cv_data = cv(
    params = params,
    pool = train_full_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False
)

Training on fold [0/5]

bestTest = 0.866360207
bestIteration = 881

Training on fold [1/5]

bestTest = 0.8716621864
bestIteration = 998

Training on fold [2/5]

bestTest = 0.8695412245
bestIteration = 540

Training on fold [3/5]

bestTest = 0.8795945701
bestIteration = 942

Training on fold [4/5]

bestTest = 0.8586794872
bestIteration = 750



In [28]:
cv_data

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.818479,0.009480,0.685020,0.000209,0.684939,0.000262
1,1,0.823848,0.010334,0.677216,0.000374,0.677055,0.000503
2,2,0.831819,0.009265,0.668989,0.000472,0.668799,0.000751
3,3,0.834559,0.008771,0.662008,0.001326,0.661753,0.000689
4,4,0.834483,0.008218,0.654599,0.001429,0.654270,0.000898
...,...,...,...,...,...,...,...
995,995,0.868703,0.007809,0.329986,0.016873,0.284097,0.003644
996,996,0.868715,0.007781,0.329972,0.016875,0.284057,0.003631
997,997,0.868718,0.007804,0.329972,0.016876,0.284003,0.003636
998,998,0.868725,0.007777,0.329969,0.016855,0.283943,0.003613


In [30]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
741,741,0.868822,0.007457,0.330818,0.017005,0.297428,0.004326


In [36]:
n_iters = cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]['iterations'].values[0]

In [40]:
n_iters

741

In [37]:
params = {'verbose': 100,
          'iterations': n_iters,
          'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          'random_seed': 42,
          'learning_rate': 0.01}

In [38]:
model = CatBoostClassifier(**params)

In [39]:
model.fit(train_full_data)

0:	total: 23.5ms	remaining: 17.4s
100:	total: 2.36s	remaining: 15s
200:	total: 4.75s	remaining: 12.8s
300:	total: 7.05s	remaining: 10.3s
400:	total: 9.46s	remaining: 8.02s
500:	total: 11.7s	remaining: 5.61s
600:	total: 14s	remaining: 3.27s
700:	total: 16.4s	remaining: 938ms
740:	total: 17.3s	remaining: 0us


In [44]:
test['y_score_cross_val'] = model.predict_proba(test_data)[:,1]

In [45]:
from sklearn.metrics import roc_auc_score

In [46]:
roc_auc_score(test.Exited, test.y_score_cross_val)

0.8737196364315009

#### Без кросс-валидации результат был 0.8719, а с CV - 0.8737
В данном случае CV не дала большого прироста, но данный подход помогает получать более точные и уверенные результаты

## Подбор гиперпараметров

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/catboost_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=2043

https://github.com/catboost/catboost/blob/master/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning.ipynb

In [47]:
model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'AUC',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 741,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntro

In [48]:
params = {'verbose': 100,
          'eval_metric': 'AUC',
          'loss_function': 'Logloss',
          'random_seed': 42}

In [49]:
model = CatBoostClassifier(**params)

In [50]:
grid = {'learning_rate': [0.01, 0.1],
        'depth': [5, 6]}

In [51]:
result = model.grid_search(grid, train_full_data, verbose=False)

0:	test: 0.7811641	best: 0.7811641 (0)	total: 17.6ms	remaining: 17.6s
100:	test: 0.8435462	best: 0.8435769 (99)	total: 2.15s	remaining: 19.1s
200:	test: 0.8508923	best: 0.8508923 (200)	total: 4.2s	remaining: 16.7s
300:	test: 0.8547897	best: 0.8547897 (300)	total: 6.24s	remaining: 14.5s
400:	test: 0.8569385	best: 0.8569385 (400)	total: 8.43s	remaining: 12.6s
500:	test: 0.8576410	best: 0.8576410 (500)	total: 10.5s	remaining: 10.5s
600:	test: 0.8585538	best: 0.8585538 (600)	total: 12.6s	remaining: 8.33s
700:	test: 0.8590923	best: 0.8591487 (694)	total: 14.4s	remaining: 6.13s
800:	test: 0.8593590	best: 0.8595564 (749)	total: 16.2s	remaining: 4.02s
900:	test: 0.8593282	best: 0.8595564 (749)	total: 18s	remaining: 1.97s
999:	test: 0.8593897	best: 0.8595974 (986)	total: 19.7s	remaining: 0us

bestTest = 0.8595974359
bestIteration = 986

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	test: 0.7811641	best: 0

In [52]:
result['params']

{'depth': 5, 'learning_rate': 0.1}

In [53]:
pd.DataFrame(result['cv_results'])['test-AUC-mean'].max()

0.8689660773549619

## Принципы sklearn

https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

## Предобработка и Фиченжениринг
Для катбуста не обязательно масштабировать признаки

https://catboost.ai/en/docs/concepts/quantization

https://www.kaggle.com/learn/feature-engineering

# Калибровка

https://www.kaggle.com/residentmario/notes-on-classification-probability-calibration

# Catboost, тексты и эмбеддинги

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/text_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=3802

# Главные навыки для аналитика

- SQL
- pandas
- Основы статистики (AB тесты)

# Главные навыки для Дата Саентиста

- Все то же, что и у аналитика
- Машинное обучение (sklearn, catboost)
- более продвинутый python (основы ООП)
- linux, git, docker, веб-сервисы

# Как развиваться дальше?

- [Прикладное машинное обучение с помощью Scikit-Learn, Keras и TensorFlow: концепции, инструменты и техники для создания интеллектуальных систем, 2-е издание | Жерон Орельен](https://www.ozon.ru/product/prikladnoe-mashinnoe-obuchenie-s-pomoshchyu-scikit-learn-keras-i-tensorflow-kontseptsii-207392052/?sh=HC92ZQyhUQ)
- Задачи на kaggle, вот [список](https://www.kaggle.com/getting-started/114864) для начинающих
- Откликайся на вакансии и решай тестовые. Можешь смотреть [тестовые по дс у меня на канале](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbefSg1XN3FA-SdSRFcCQfn)
- [Мой курс по статистике](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbHLLI66KdscKp_FJt0FsIi)
- Задачи по pandas и sql на [StrataScratch](https://www.stratascratch.com/?via=gleb)
- [Мой курс по SQL](https://www.udemy.com/course/sql-with-gleb/?referralCode=C99EC81AE75FEC9F50A2)
- Задачи на [itresume](https://itresume.ru/)
- Задачи на [Checkio](https://checkio.org/)
- Задачи на [Leetcode](https://leetcode.com/) вот по этому [списку](https://seanprashad.com/leetcode-patterns/)
- Мой [плейлист](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xZ4qiXlmQQLgAPyXJhQ7HxZ) по основам Linux
- [Основы Git](https://youtu.be/0cGIiA0AjNw)
- [Основы докера](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xbR0ka2TUxJkXna40XWu92m)
- [Основы веб-сервисов](https://www.youtube.com/playlist?list=PLQJ7ptkRY-xYLEAC5Y_sKqrJ9RA-U7Dja)

# Дополнительные материалы

- [Как работает градиентный бустинг](https://youtu.be/ZNJ3lKyI-EY)
- [Как учить английский](https://youtu.be/f9Q4Mwvd5pc)
- [Как искать работу](https://deepnote.com/@gleb-mikhaylov-15d6/blog-posts-nXDqO5CTQYOreNE53H3y-A)